# Goal

Forecast the amplitude for turning the relic step on/off in the RSD term for a degenerate neutrino cosmology. We want to see if this matches or disagrees with the result given by Linda's MCMCs w/, w/o  relicfast. 

# Logistics

None

# Setup Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment (we use Python 3.6.8) there if you'd like: 
```
$ cd .../<project-directory>
$ python -m virtualenv env 
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you:
```
$ python 
>>> import cosmicfish as cf
>>> cf.install_class('<project-directory>/class')
>>> exit()
```

# Import & Configure _cosmicfish_

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
import os
import shutil
import scipy
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/cfworkspace/")
datastore = cf.correct_path("~/Desktop/datastore/")
classpath = os.path.join(projectdir, "class")

Specify the granularity of numerical derivatives.

In [ ]:
derivative_step = 0.01 #How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 #For calculating numerical integral wrt mu between -1 and 1 

# Specify Fiducial Cosmologies

In [ ]:
#Linda Fiducial
#neutrinofid = {
#        "A_s" : 2.2321e-9, 
#        "n_s" : 0.96659,
#        "omega_b" : 0.02226,
#        "omega_cdm" : 0.11271,
#        "tau_reio" : 0.059888,
#        "h" : 0.70148,
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : 0.03, # Units [eV]
#        "b0" : 1.0, 
#        "alphak2" : 1.0,
#        "sigma_fog_0" : 250000, #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None #Not used for neutrino forecasts
#        } 

# DESI No Relic Steps MCMC Bestfit Values
neutrinofid = { #Applying forecast shift from no step 
        "A_s" : 2.231027e-09, 
        "n_s" : 9.662029e-01,
        "omega_b" : 2.226660e-02,
        "omega_cdm" : 1.126965e-01,
        "tau_reio" : 5.944537e-02,
        "h" : 7.008683e-01,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 0.0364634, # Units [eV]
        "b0" : 1.004659e+00, 
        "alphak2" : 9.794683e-01,
        "sigma_fog_0" : 2.496839e+05, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : None #Not used for neutrino forecasts
        } 

# DESI Shift Forecast Applied to Above No Relic Steps Fiducials
#neutrinofid = { #Applying forecast shift from no step 
#        "A_s" : (2.231027e-09 + 6.483268e-12), 
#        "n_s" : (9.662029e-01 + 5.847986e-05),
#        "omega_b" : (2.226660e-02 - 1.092514e-07),
#        "omega_cdm" : (1.126965e-01 - 2.240456e-05),
#        "tau_reio" : (5.944537e-02 + 1.525700e-03),
#        "h" : (7.008683e-01 - 7.350068e-04),
#        "T_cmb" : 2.726, # Units [K]
#        "N_ncdm" : 3., 
#        "T_ncdm" : 1.95/2.726, 
#        "m_ncdm" : (0.0364634 + 0.002999569333), # Units [eV]
#        "b0" : (1.004659e+00 + 4.432180e-03), 
#        "alphak2" : (9.794683e-01 - 3.124481e-02),
#        "sigma_fog_0" : (2.496839e+05 - 9.316495e+01), #Units [m s^-2]
#        "N_eff" : 3.046 - (3 * 1.0132), 
#        "relic_fix" : None #Not used for neutrino forecasts
#        } 


# Specify Experiment Observational Parameters

In [ ]:
#Specify redshift bins, noise per bin, and sky coverage
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `convergence` class of `cosmicfish`. 

We pass to `convergence` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology by step sizes specified by the user and compute the corresponding power spectrum derivatives. 

All 'varyfactors' are computed relative to the fiducial cosmology. That is:

dtheta = varyfactor * theta_fiducial

**WARNING: This process takes a considerable amount of time (~5 mins per varyfactor w/o prexisting data). Only run when you need to.**

In [ ]:
neutrinoconvergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
    neutrinofid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    mu_step=mu_integral_step,
    varyfactors=[0.003, 0.004, 0.005, 0.006, 0.007] # Relative factors used to compute convergence
    )
neutrinoconvergencetest.gen_all_plots() # Display convergence plots

# Run Forecasts

In [ ]:
neutrinoforecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    neutrinofid, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

neutrinoforecast.gen_pm()

neutrinoforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                    
        #'omega_ncdm',                                 
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2',
        #'D_Amp'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)

In [ ]:
pd.DataFrame(neutrinoforecast.fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
        #'D_Amp'
    ]).to_csv("/Users/nicholasdeporzio/Desktop/extendedlssfisher.csv", sep='\t')
pd.DataFrame(neutrinoforecast.fisher)

In [ ]:
DD = np.array(neutrinoforecast.fisher[10, 10])
Dj = np.array(neutrinoforecast.fisher[0:10, 10])
#pd.DataFrame(Dj)
print(np.sqrt(DD))

In [ ]:
Extended_Fisher = np.array(neutrinoforecast.fisher)
neutrinoforecast.fisher = np.array(neutrinoforecast.fisher[0:10, 0:10])
neutrinoforecast.fisher_order.remove('D_Amp')

In [ ]:
#pd.DataFrame(neutrinoforecast.fisher)

In [ ]:
# Set LSS information on tau_reio to zero 
#neutrinoforecast.fisher[:,4] = 0.
#neutrinoforecast.fisher[4,:] = 0.

# Add CMB Data

In [ ]:
neutrinoforecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'M_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

# Save Fisher Matrices

In [ ]:
neutrinoforecast.export_matrices("~/Desktop")

In [ ]:
pd.DataFrame(neutrinoforecast.numpy_full_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]).to_csv("/Users/nicholasdeporzio/Desktop/yescmbfisher.csv", sep='\t')
pd.DataFrame(neutrinoforecast.numpy_full_fisher)

In [ ]:
pd.DataFrame(neutrinoforecast.numpy_lss_fisher, columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                          
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]).to_csv("/Users/nicholasdeporzio/Desktop/nocmbfisher.csv", sep='\t')
pd.DataFrame(neutrinoforecast.fisher)

# Compute Shift Vector

In [ ]:
# Print the uncertainty in the P_g shift amplitude
D_spread = np.sqrt(DD)
print(r"Uncertainty in P_g shift amplitude: ", D_spread)

In [ ]:
#Djvec1=(neutrinoforecast.numpy_full_fisher[8,:]  *  0.001)

In [ ]:
#neutrinoforecast.pandas_full_covariance

In [ ]:
for i in range(10): 
    print(np.sqrt(neutrinoforecast.numpy_full_covariance[i, i]), '\n')

In [ ]:
#Amp_Projection[4] = 0.
print("Amplitude vector, <D, g_i>:")
pd.DataFrame(np.array(Dj))

In [ ]:
Delta_A = np.matmul(np.linalg.inv(np.array(neutrinoforecast.numpy_full_fisher)), Dj)
#Delta_A = np.matmul(neutrinoforecast.numpy_full_covariance, Djvec1)

print('Parameter shift vector, delta a_i: ')
pd.DataFrame(np.array(Delta_A))

In [ ]:
print("Parameter shift/error vector, (delta a_i / sigma_i): ")
params = [
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ]
for i in range(10): 
    print("delta a_i / sigma_i for", params[i], ": ", (Delta_A[i]
                                                       /np.sqrt(np.linalg.inv(np.array(neutrinoforecast.numpy_full_fisher))[i,i])))

In [ ]:
print("Fisher Matrix: ")
pd.DataFrame(np.array(neutrinoforecast.fisher), columns=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                    
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ])

# Plot Fisher Ellipses on MCMC Contour

In the last step, we exported three Fisher matrices. The next steps are performed using MontePython outside this notebook:

```
$ mv ~/Desktop/inv_fullfisher.dat <project-directory>/chains/inv_fisher.dat
$ cd <project-directory>
$ ./montepython_public/montepython/MontePython.py info chain/ --plot-fisher --center-fisher
```

Then, the desired triangle plot is located in <project-directory>/chains/plots/triangle.pdf

In [ ]:
neutrinoforecast.psterms

# Compare Pg with MCMC Values

In [ ]:
Linda_yes_relicfast = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/other/Pg_90meV_DH_yes_Relicfast.dat", skiprows=1, usecols=(0, 3, 5))
Linda_no_relicfast = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/other/Pg_90meV_DH_no_Relicfast.dat", skiprows=1, usecols=(0, 3, 5))

Linda_Pg_nostep_ktable = Linda_no_relicfast[0:100, 0] * 0.70148
Linda_Pg_nostep = Linda_no_relicfast[0:100, 1]
Linda_bias_nostep = Linda_no_relicfast[0:100, 2]

Linda_Pg_yesstep_ktable = Linda_yes_relicfast[0:100, 0] * 0.70148
Linda_Pg_yesstep = Linda_yes_relicfast[0:100, 1]
Linda_bias_yesstep = Linda_yes_relicfast[0:100, 2]

ktab = np.geomspace(np.min(Linda_Pg_yesstep_ktable), np.max(neutrinoforecast.k_table[0]), 100)

Linda_yes_interp = np.interp(ktab, Linda_Pg_yesstep_ktable, Linda_Pg_yesstep)
Nick_yes_interp = np.interp(ktab, neutrinoforecast.k_table[0], neutrinoforecast.Pg[0, :, 20])

Linda_no_interp = np.interp(ktab, Linda_Pg_nostep_ktable, Linda_Pg_nostep)
Nick_no_interp = np.interp(ktab, neutrinoforecast.k_table[0], neutrinoforecast.Pg_norelicstep[0, :, 20])

Linda_yes_bias_interp = np.interp(ktab, Linda_Pg_yesstep_ktable, Linda_bias_yesstep)
Nick_yes_bias_interp = np.interp(ktab, neutrinoforecast.k_table[0], np.sqrt(neutrinoforecast.RSD[0, :, 20]))

Linda_no_bias_interp = np.interp(ktab, Linda_Pg_nostep_ktable, Linda_bias_nostep)
Nick_no_bias_interp = np.interp(ktab, neutrinoforecast.k_table[0], np.sqrt(neutrinoforecast.RSD_norelicstep[0, :, 20]))

Linda_Delta_P = ((Linda_yes_interp - Linda_no_interp)/Linda_yes_interp)
Nick_Delta_P = ((Nick_yes_interp - Nick_no_interp)/Nick_no_interp)

In [ ]:
# Plot D amplitude 
plt.figure(figsize=(15,7.5))
plt.semilogx(ktab, Nick_Delta_P, Label="Fisher Forecast")
plt.semilogx(ktab, Linda_Delta_P, Label="MCMC Analysis")
plt.title(r"Normalized $P_g$ Shift")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"$(P_g(\Delta L=0.6) - P_g(\Delta L=0)) / P_g(\Delta L =0.6)$")
plt.legend()
plt.savefig("/Users/nicholasdeporzio/Desktop/D_shift.png")
plt.show()

# Plot D amplitude difference
plt.figure(figsize=(15,7.5))
plt.semilogx(ktab, Linda_Delta_P-Nick_Delta_P)
plt.title(r"Difference in Normalized $P_g$ Shift ,MCMC-Forecast ")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"Normalized $P_g$ Step Difference")
plt.savefig("/Users/nicholasdeporzio/Desktop/D_shift_Difference.png")
plt.show()

# Plot D amplitude relative difference
plt.figure(figsize=(15,7.5))
plt.semilogx(ktab, (Linda_Delta_P-Nick_Delta_P)/Linda_Delta_P)
plt.title(r"Normalized Difference in Normalized $P_g$ Shift ,(MCMC-Forecast)/MCMC ")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"Normalized $P_g$ Step Difference")
plt.savefig("/Users/nicholasdeporzio/Desktop/D_shift_Relative_Difference.png")
plt.show()

# Plot Pg no relic step
plt.figure(figsize=(15,7.5))
plt.loglog(ktab, Nick_no_interp, Label="Fisher Forecast")
plt.loglog(ktab, Linda_no_interp, Label="MCMC Analysis")
plt.title(r"$P_g$ No Relic Step")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"$P_g(\Delta L=0)$")
plt.legend()
plt.savefig("/Users/nicholasdeporzio/Desktop/Pg_nostep.png")
plt.show()

# Plot Pg no relic step ratio
plt.figure(figsize=(15,7.5))
plt.loglog(ktab,  Linda_no_interp/Nick_no_interp)
plt.title(r"$P_g$ No Relic Step Ratio, MCMC/Forecast")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"P_g (MCMC/Forecast)")
plt.savefig("/Users/nicholasdeporzio/Desktop/Pg_nostep_ratio.png")
plt.show()

# Plot Pg yes relic step
plt.figure(figsize=(15,7.5))
plt.loglog(ktab, Nick_yes_interp, Label="Fisher Forecast")
plt.loglog(ktab, Linda_yes_interp, Label="MCMC Analysis")
plt.title(r"$P_g$ Yes Relic Step")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"$P_g(\Delta L=0.6)$")
plt.legend()
plt.savefig("/Users/nicholasdeporzio/Desktop/Pg_yesstep.png")
plt.show()

# Plot Pg yes relic step ratio
plt.figure(figsize=(15,7.5))
plt.loglog(ktab,  Linda_yes_interp/Nick_no_interp)
plt.title(r"$P_g$ Yes Relic Step Ratio, MCMC/Forecast")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"$P_g (MCMC/Forecast)$")
plt.savefig("/Users/nicholasdeporzio/Desktop/Pg_yesstep_ratio.png")
plt.show()


#Compare bias terms with step
plt.figure(figsize=(15,7.5))
plt.loglog(ktab, Nick_yes_bias_interp, label="Fisher Bias")
plt.loglog(ktab,  Linda_yes_bias_interp, label="MCMC Bias")
plt.title(r"Bias Term Comparison, Yes Relic Step")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"Bias")
plt.legend()
plt.savefig("/Users/nicholasdeporzio/Desktop/bias_comparison_yesstep.png")
plt.show()

#Compare bias terms with no step
plt.figure(figsize=(15,7.5))
plt.loglog(ktab, Nick_no_bias_interp, label="Fisher Bias")
plt.loglog(ktab,  Linda_no_bias_interp, label="MCMC Bias")
plt.title(r"Bias Term Comparison, No Relic Step")
plt.xlabel("k [Mpc^-1]")
plt.ylabel(r"Bias")
plt.legend()
plt.savefig("/Users/nicholasdeporzio/Desktop/bias_comparison_nostep.png")
plt.show()

In [ ]:
print(neutrinoforecast.spectra_mid[0].D)
print(neutrinoforecast.b0_fid)
print(cf.bL(neutrinoforecast.b0_fid, neutrinoforecast.spectra_mid[0].D))
print(np.power(neutrinoforecast.k_table[0][50], 2))
print(neutrinoforecast.Pm[0][50])
print(neutrinoforecast.alphak2_fid)

In [ ]:
import dill
dill.dump_session('/Users/nicholasdeporzio/Desktop/relicfast_shift_forecast.db')

In [ ]:
dill.load_session('relicfast_shift_forecast.db')